In [1]:
import os
import sys
import tensorflow as tf
import numpy as np
from keras.models import Model
from keras.layers import Input, Dense, LSTM, Lambda
from keras.engine.topology import Input
from keras import optimizers
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout
from keras.layers import Activation, BatchNormalization, MaxPooling2D
import time
import math
from keras.utils import plot_model
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.engine.topology import Input
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras import backend as K
K.set_image_dim_ordering('tf')

import gazenetGenerator as gaze_gen

/Users/ouyangjiahong/tensorflow/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [23]:
# global param
dataset_path = '../gaze_dataset'
learning_rate = 0.0001
time_steps = 32
num_classes = 6
batch_size = 4
origin_image_size = 360    # size of the origin image before the cropWithGaze
img_size = 128    # size of the input image for network
num_channel = 3
steps_per_epoch=400
epochs=100
validation_step=20

In [15]:
class GazeNet():
    def __init__(self,learning_rate,time_steps,num_classes,batch_size):
        self.learning_rate = learning_rate
        self.time_steps = time_steps
        self.num_classes = num_classes
        self.batch_size = batch_size
        self.kernel_size = 15
        self.kernel_num = 256
        self.gaussian_sigma = 1
        self.gaussian_weight = self.create_gaussian_weight()
        self.model = self.create_model()

    def create_gaussian_weight(self):
        kernel_size = self.kernel_size    #same with the shape of the layer before flatten
        kernel_num = self.kernel_num
        r = (kernel_size - 1) // 2
        sigma_2 = float(self.gaussian_sigma * self.gaussian_sigma)
        pi = 3.1415926
        ratio = 1 / (2*pi*sigma_2)

        kernel = np.zeros((kernel_size, kernel_size))
        for i in range(-r, r+1):
            for j in range(-r, r+1):
                tmp = math.exp(-(i*i+j*j)/(2*sigma_2))
                kernel[i+r][j+r] = round(tmp, 3)
        kernel *= ratio
        kernel = np.expand_dims(kernel, axis=2)
        kernel = np.tile(kernel, (1,1,kernel_num))
        # print(kernel.shape)
        return kernel

    def create_model(self):

        model = Sequential()

        #block 1
        model.add(Conv2D(96,(5,5),strides = (2,2),
                            padding = 'valid',
                            activation = 'relu',
                            input_shape = (128,128,3)
                            ))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size = (2,2)))

        #block 2
        model.add(Conv2D(256,(3,3),padding = 'same'))
        model.add(BatchNormalization())
        model.add(Activation('relu'))

        model.add(Conv2D(256,(3,3),padding = 'same'))
        model.add(BatchNormalization())
        model.add(Activation('relu'))

        model.add(Conv2D(256,(3,3),padding = 'same'))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(MaxPooling2D(2,2))

        def multiply_constant(input):
            for i in range(self.batch_size*self.time_steps):
                tmp = tf.multiply(tf.cast(input[i], tf.float32), tf.cast(self.gaussian_weight, tf.float32))
                tmp = tf.expand_dims(tmp, 0)
                if i == 0:
                    res = tmp
                else:
                    res = tf.concat([res, tmp], 0)
            res = tf.reshape(res,[self.batch_size,self.time_steps,
                                  self.kernel_size,self.kernel_size,self.kernel_num])
            res = tf.reshape(res,[self.batch_size,self.time_steps,
                                  self.kernel_size*self.kernel_size*self.kernel_num])
            return res

        model.add(Lambda(multiply_constant))

        def mean_value(input):
            return tf.reduce_mean(input,1)

        model.add(LSTM(128,return_sequences = True))
        model.add(LSTM(6,return_sequences = True))
        model.add(Lambda(mean_value))

        adam = optimizers.Adam(lr = self.learning_rate)
        model.compile(loss='categorical_crossentropy', optimizer='adam')
        print(model.summary())

        return model
    
#     def train(self):
#         # categorical_labels = to_categorical(int_labels, num_classes=None)
#         pass

#     def load_data(self):
#         print("Hello world")

#     def save_model_weights(self, folder_path, suffix):
#         # Helper function to save your model / weights.
#         self.model.save_weights(folder_path + 'weights-' +  str(suffix) + '.h5')
#         self.model.save(folder_path + 'model-' +  str(suffix) + '.h5')

#     def load_model(self, model_file):
#         # Helper function to load an existing model.
#         self.model = load_model(model_file)

#     def load_model_weights(self,weight_file):
#         # Helper funciton to load model weights.
#         self.model.load_weights(weight_file)

In [16]:
def cropWithGaze(images,gazes,batch_size,time_steps,img_size,num_channel):
    img_seq = np.zeros(batch_size,time_steps,img_size,img_size,num_channel)
    for i in range(batch_size):
        for j in range(time_steps):
            x = gaze[1]
            y = gaze[2]
            image_size = images.shape[2]
            assert image_size > img_size
            right_bound = min(x+(img_size/2),image_size)
            left_bound = max(0,x-(img_size/2))
            up_bound = max(0,y-(img_size/2))
            down_bound = min(image_size,y+(img_size/2))
            img_seq[batch_size,time_steps,:,:,:] = images[batch_size,time_steps,up_bound:down_bound,left_bound:right_bound,:]
    return img_seq

In [24]:
def main(args):
    # generate model
    gaze_net = GazeNet(learning_rate,time_steps,num_classes,batch_size)
    model = gaze_net.model
    plot_model(model, to_file='model.png')
    print("generate model!")
    
    # generatr generator
    trainGenerator = gaze_gen.GazeDataGenerator(validation_split=0.2)
    train_data = trainGenerator.flow_from_directory(dataset_path, subset='training',time_steps=time_steps, 
                                                    batch_size=batch_size, crop=True, gaussian_std=0.01)
    val_data = trainGenerator.flow_from_directory(dataset_path, subset='validation', time_steps=time_steps, 
                                                  batch_size=batch_size, crop=True, gaussian_std=0.01)
    # [img_seq, gaze_seq], output = next(trainGeneratorDirectory)
    print("fetch data!")
    
    # start training
    checkpoint = ModelCheckpoint('weight.{epoch:02d}.hdf5', monitor='val_acc', mode='max', period=5)
    model.fit_generator(train_data, steps_per_epoch=steps_per_epoch, epochs=epochs, callbacks=[checkpoint], 
                    validation_data=val_data, validation_step=validation_step, shuffle=False)
    print("finished training!")
    
    
#     [img_seq, gaze_seq], output = next(trainGeneratorDirectory)
#     img_seq = cropWithGaze(img_seq,gaze_seq,batch_size,time_steps,img_size,num_channel)


In [25]:
if __name__ == '__main__':
    main(sys.argv)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 62, 62, 96)        7296      
_________________________________________________________________
batch_normalization_17 (Batc (None, 62, 62, 96)        384       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 31, 31, 96)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 31, 31, 256)       221440    
_________________________________________________________________
batch_normalization_18 (Batc (None, 31, 31, 256)       1024      
_________________________________________________________________
activation_13 (Activation)   (None, 31, 31, 256)       0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 31, 31, 256)       590080    
__________

TypeError: fit_generator() got an unexpected keyword argument 'validation_step'